In [1]:
# Download data
%run "../dataio/gtsrb_download.py"

ModuleNotFoundError: No module named 'logger'

In [2]:
# Hard coded because I can't be bothered to make it dynamic LMAO
%cd c:\Users\Gebruiker\Documents\YEAR 3\Neural Networks\traffic-sign-classification

[Errno 2] No such file or directory: 'c:UsersGebruikerDocumentsYEAR 3Neural Networkstraffic-sign-classification'
/home/marcusp/Documents/traffic-sign-classification/notebooks


In [3]:
from dataio.transforms import ToCompose, ToResize, ToRotate, ToNoise, ToTensor, ToNormalize
from dataio.gtsrb_dataset import GTSRBDataset
from dataio.dataloader import DataLoader
from nn.layers.batchnorm2d import BatchNorm2D
from nn.layers.conv2d import Conv2D
from nn.layers.dropout import Dropout
from nn.layers.flatten import Flatten
from nn.layers.linear import Linear
from nn.layers.maxpool2d import MaxPool2D
from nn.layers.sequential import Sequential
from nn.optim import Adam
from nn.loss import cross_entropy
from train import train
from crossval import cross_validate
import numpy as np

ModuleNotFoundError: No module named 'dataio'

In [ ]:
# Define the transforms for training
train_transforms = ToCompose([
    ToResize(size=64),
    ToRotate(angle=15),
    ToNoise(mean=0, std=0.05),
    ToTensor(),
    ToNormalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

# Define the transforms for validation and testing
val_transforms = ToCompose([
    ToResize(size=64),
    ToTensor(),
    ToNormalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

# Define the desired classes
desired_classes = {0, 12, 17, 20, 30}

# Create a temporary dataset instance to access all labels
temp_dataset = GTSRBDataset(root="./data/gtsrb/", split="train", indices=[])
all_labels = temp_dataset.labels_data  # This contains all the labels

# Now filter the indices based on all_labels
desired_indices = [i for i, (_, label) in enumerate(all_labels) if label in desired_classes]

# Now define the splits based on desired_indices
def get_train_indices():
    return desired_indices[:int(0.7 * len(desired_indices))]

def get_val_indices():
    start = int(0.7 * len(desired_indices))
    end = int(0.85 * len(desired_indices))
    return desired_indices[start:end]

def get_test_indices():
    start = int(0.85 * len(desired_indices))
    return desired_indices[start:]

# Initialize the dataset
train_dataset = GTSRBDataset(
    root="./data/gtsrb/",
    indices=get_train_indices(),
    split="train",
    transforms=train_transforms
)
val_dataset = GTSRBDataset(
    root="./data/gtsrb/",
    indices=get_val_indices(),
    split="val",
    transforms=val_transforms
)
test_dataset = GTSRBDataset(
    root="./data/gtsrb/",
    indices=get_test_indices(),
    split="test",
    transforms=val_transforms
)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [ ]:
# Example architecture without ReLU
layers = [
    Conv2D(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1),
    BatchNorm2D(num_channels=32),
    MaxPool2D(pool_size=2, stride=2),
    Flatten(),
    Linear(in_features=32 * 32 * 32, out_features=512),  # Adjust input_size based on your image size and pooling layers
    Dropout(p=0.5),
    Linear(in_features=512, out_features=43)  # GTSRB has 43 classes
]

model = Sequential(layers)

In [ ]:
# Get the parameters from the model
param_dict = {}
for layer in model.layers:
    param_list = layer.params()
    if isinstance(param_list, dict):
        param_dict.update(param_list)
    elif isinstance(param_list, list):
        for i, param in enumerate(param_list):
            name = f"layer_{layer}_{i}"
            param_dict[name] = param
    else:
        name = f"layer_{layer}"
        param_dict[name] = param_list

# Initialize the optimizer (e.g., SGD, Adam, or Momentum)
optimizer = Adam(param_dict, lr=0.001)  # Example using Adam optimizer

# Define the loss function
loss_fn = cross_entropy


In [ ]:
from train import train

# Train the model using the train function
train_losses, val_losses, train_accs, val_accs = train(
    model, train_loader, val_loader, loss_fn, optimizer, num_epochs=10
)

In [ ]:
results = cross_validate(
    model=model,
    optimizer=optimizer,
    loss_fn=loss_fn,
    checkpoint_root="./checkpoints/crossval/",
    log_dir="./logs/crossval/"
)

In [ ]:
import importlib
import evaluate

importlib.reload(evaluate)  # reloads the updated file

test_loss, test_acc = evaluate.evaluate(model, test_loader, loss_fn)

In [ ]:
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

In [ ]:
import viz
importlib.reload(viz)  # reloads the updated file


viz.plot_curves(train_losses, val_losses, train_accs, val_accs)

In [ ]:


for batch_data, batch_labels in test_loader:
    predictions = np.argmax(model.forward(batch_data), axis=1)
    viz.plot_confusion_matrix(batch_labels, predictions, desired_classes)
    break  # Just plot the first batch for demonstration



In [ ]:
importlib.reload(viz)
# Compute and plot saliency maps one sample at a time
for batch_data, batch_labels in test_loader:
    for i, input_sample in enumerate(batch_data):
        input_sample = np.array(input_sample, dtype=float)
        saliency_map = compute_saliency_map(model, input_sample, batch_size=1000)  # Use batch_size as needed
        plot_saliency_map(saliency_map)

        if i >= 3:  # just first 4 samples for demonstration
            break
    break  # first batch only
